![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

In [3]:
import pandas as pd
import plotly.express as px
import json
import boto3
import ast
import folium
import requests
from io import BytesIO
from branca.element import Figure
from sqlalchemy import create_engine, text

In [4]:
cities = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris",
    "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar",
    "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille",
    "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
    "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege",
    "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

In [5]:
def get_coordinates(city):
    # Replace spaces with '+' characters
    city = city.replace(' ', '+')
    
    # Set the query URL with the specified city
    url = f"https://nominatim.openstreetmap.org/search?q={city}&format=json"
    
    headers = {
        'User-Agent': 'my-app/0.0.1'  # Replace with your app name and version
    }

    try:
        # Executes GET request to Nominatim API with headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Get JSON data from response
        data = response.json()

        if data:
            # Extract latitude and longitude from the first result
            lat = data[0]['lat']
            lon = data[0]['lon']
            return lat, lon
        else:
            print(f"No coordinates found for {city}")
            return None, None
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching coordinates for {city}: {e}")
        return None, None

# Example usage:
city = "New York"
latitude, longitude = get_coordinates(city)
if latitude and longitude:
    print(f"The coordinates of {city} are: Latitude {latitude}, Longitude {longitude}")
else:
    print(f"Failed to retrieve coordinates for {city}")


The coordinates of New York are: Latitude 40.7127281, Longitude -74.0060152


In [6]:
# Creating a DataFrame to store GPS coordinate information
coordinates_df = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

# Loop through the list of cities to get coordinates for each city
for city in cities:
    lat, lon = get_coordinates(city)
    new_row = pd.DataFrame({
        'City': [city],
        'Latitude': [lat],
        'Longitude': [lon]
    })
    coordinates_df = pd.concat([coordinates_df, new_row], ignore_index=True)

coordinates_df

,City,Latitude,Longitude
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,49.314695,-96.9538228
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658
5,Paris,48.8534951,2.3483915
6,Amiens,49.8941708,2.2956951
7,Lille,50.6365654,3.0635282
8,Strasbourg,48.584614,7.7507127
9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503


In [7]:
# API key obtained following registration on the site
api_key = "6005eb181c0e673b5a45954c37cc6b79" # Your Open-weather API key

# Function to retrieve weather data using the OpenWeatherMap API 
def get_weather(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        return weather_data
    else:
        print(f"Failed to get weather data: Status code {response.status_code}, Response: {response.text}")
        return None
    
# Applies the function to each line of `coordinates_df` to retrieve detailed weather data
coordinates_df['Weather'] = coordinates_df.apply(lambda row: get_weather(row['Latitude'], row['Longitude']), axis=1)
coordinates_df

,City,Latitude,Longitude,Weather
0,Mont Saint Michel,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
1,St Malo,49.314695,-96.9538228,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
2,Bayeux,49.2764624,-0.7024738,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
3,Le Havre,49.4938975,0.1079732,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
4,Rouen,49.4404591,1.0939658,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
5,Paris,48.8534951,2.3483915,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
6,Amiens,49.8941708,2.2956951,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
7,Lille,50.6365654,3.0635282,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
8,Strasbourg,48.584614,7.7507127,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."
9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'..."


In [8]:
# Data transformation into csv file
coordinates_df.to_csv('coordinates.csv', index=False)

In [9]:
def weather(weather_data):
    total_temp = 0
    total_precipitation = 0
    
    if 'list' in weather_data:
        for day in weather_data['list']:
            # Conversion de Kelvin en Celsius pour la température
            temp_celsius = day['main']['temp'] - 273.15
            total_temp += temp_celsius
            
            # Ajout des précipitations en millimètres s'il y en a
            if 'rain' in day and '3h' in day['rain']:
                total_precipitation += day['rain']['3h']
        
        # Calcul de la moyenne de la température en Celsius
        if len(weather_data['list']) > 0:
            average_temp = total_temp / len(weather_data['list'])
        else:
            average_temp = 0
    else:
        average_temp = 0
        total_precipitation = 0
    
    return average_temp, total_precipitation


In [10]:
coordinates_df[['AverageTemp', 'TotalPrecipitation']] = coordinates_df.apply(lambda row: weather(row['Weather']), axis=1, result_type='expand')

In [11]:
coordinates_df.head()

,City,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
1,St Malo,49.314695,-96.9538228,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",17.42875,6.89
2,Bayeux,49.2764624,-0.7024738,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.03350,7.33
3,Le Havre,49.4938975,0.1079732,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.48325,7.65
4,Rouen,49.4404591,1.0939658,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",19.13275,7.31


In [12]:
#!python scraping.py

In [13]:
# Reading the Json file recovered during scraping
booking_df = pd.read_json('hotels.json')
booking_df.head()

,City,Hotel Name,Hotel URL,Score,Description,Latitude,Longitude
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396


In [14]:
# Renaming columns for clarity and avoid confusion when merging data
booking_df.rename(columns={'Latitude': 'lat_hotels', 'Longitude': 'lon_hotels'}, inplace=True)
booking_df.head()

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396


In [15]:
# Data Merge
df_full = pd.merge(booking_df, coordinates_df, on='City')
df_full.head()

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.6359541,-1.511459954959514,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34


In [16]:
# saving final data to a csv file
df_full.to_csv('df_full.csv', index=False)

--- Plot of the 5 best destinations in France according to temperature ---

In [17]:
df_temp = df_full[['City', 'AverageTemp', 'Latitude', 'Longitude', 'TotalPrecipitation']]

df_temp_sorted = df_temp.sort_values(by='AverageTemp', ascending=False)

df_temp_unique = df_temp_sorted.drop_duplicates(subset=['City'])

top_5_cities = df_temp_unique.head(5)

# Affichage du top 5 des villes avec la température la plus élevée
print("Top 5 cities with the highest temperature:")
print(top_5_cities)

Top 5 cities with the highest temperature:
                City  AverageTemp    Latitude  Longitude  TotalPrecipitation
550  Aix en Provence     28.48825  43.5298424  5.4474738                0.00
612            Nimes     27.90900  43.8374249  4.3600687                0.00
528          Avignon     27.49800  43.9492493  4.8059012                0.81
603             Uzes     26.81250  44.0121279  4.4196718                0.00
777      Carcassonne     26.51625  43.2130358  2.3491069                1.10


In [18]:

# Creates a figure to contain the map with a title
cities_fig = Figure(width=800, height=400)
cities_fig.html.add_child(folium.Element("<h3 style='text-align:center;margin-top:20px;'>Top 5 des villes avec la meilleure météo</h3>"))

# Create a map centered around France
map_france = folium.Map(location=[46.2276, 2.2137], zoom_start=6, tiles='OpenStreetMap')
cities_fig.add_child(map_france)

# Ajoute des marqueurs pour les top 5 destinations
for _, row in top_5_cities.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['City']}: Temp {row['AverageTemp']}°C, Precip {row['TotalPrecipitation']}mm",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_france)

# Shows the map with the title
cities_fig

--- Top 20 des hotels pour chacune des villes ---

In [19]:
# Replaces NaNs in 'Score' column with 0 and eliminates rows with NaN coordinates
df_full['Score'].fillna(0, inplace=True)
df_full.dropna(subset=['lat_hotels', 'lon_hotels'], inplace=True)

# Filter hotels to only keep those in the top 5 cities
top_cities = top_5_cities['City'].tolist()
filtered_hotels = df_full[df_full['City'].isin(top_cities)]

# Selects the top 20 hotels for each city based on score
top_hotels = filtered_hotels.groupby('City').apply(lambda x: x.nlargest(20, 'Score')).reset_index(drop=True)

C:\Users\Rafik\AppData\Local\Temp\ipykernel_27632\3763453713.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full['Score'].fillna(0, inplace=True)
C:\Users\Rafik\AppData\Local\Temp\ipykernel_27632\3763453713.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after grou

In [20]:
# Create a new figure for the hotel map
hotel_fig = Figure(width=800, height=400)
hotel_fig.html.add_child(folium.Element("<h3 style='text-align:center;margin-top:20px;'>Top 20 des hotels pour chacune des villes</h3>"))

# Create a map centered around France
hotel_map = folium.Map(location=[46.2276, 2.2137], zoom_start=6, tiles='OpenStreetMap')
hotel_fig.add_child(hotel_map)

# Adds markers for selected hotels
for _, hotel in top_hotels.iterrows():
    folium.Marker(
        location=[hotel['lat_hotels'], hotel['lon_hotels']],
        popup=f"{hotel['Hotel Name']}: Score {hotel['Score']}<br><a href='{hotel['Hotel URL']}' target='_blank'>Book Now</a>",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(hotel_map)

# Shows the map with hotels
hotel_fig

In [21]:
# Loads the S3 session and necessary resources # Your AWS access key 
session = boto3.Session(
    aws_access_key_id= # Your AWS access key , 
    aws_secret_access_key=  # Your AWS secret key 
)
s3 = session.resource('s3')

In [22]:
# Upload the file to S3
s3.Bucket('rafik-001').upload_file('df_full.csv', 'data/df_full.csv')

In [23]:
# Downloading and reading a CSV file from an S3 bucket
try:
    obj = s3.Object('rafik-001', 'data/df_full.csv')
    df_s3 = pd.read_csv(BytesIO(obj.get()['Body'].read()))
    print("File downloaded successfully")
except Exception as e:
    print(e)

File downloaded successfully


In [24]:
# Sanity check of df
df_s3.head()

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",18.61775,8.34


In [25]:
df_s3[['City', 'Hotel Name', 'Hotel URL', 'Score', 'Description', 'lat_hotels','lon_hotels', 'Latitude', 'Longitude', 'AverageTemp','TotalPrecipitation']]

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,AverageTemp,TotalPrecipitation
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.511460,18.61775,8.34
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.511460,18.61775,8.34
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.511460,18.61775,8.34
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.511460,18.61775,8.34
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.511460,18.61775,8.34
...,...,...,...,...,...,...,...,...,...,...,...
871,Bayonne,Studio avec parking à 12mns du centre historiq...,https://www.booking.com/hotel/fr/a-12mns-du-ce...,NaN,Managed by a private host,43.485403,-1.487549,43.494514,-1.473666,20.69975,0.62
872,La Rochelle,360 degrés sur la rochelle,https://www.booking.com/hotel/fr/360-degres-su...,9.1,Managed by a private host,46.166189,-1.146359,46.159113,-1.152043,19.45650,4.64
873,La Rochelle,La Belle Amarre-Bed and Breakfast-Maison d'Hôtes,https://www.booking.com/hotel/fr/la-belle-amar...,9.4,Managed by a private host,46.158662,-1.155056,46.159113,-1.152043,19.45650,4.64
874,La Rochelle,Hôtel La Fabrique,https://www.booking.com/hotel/fr/la-fabrique-l...,8.4,Hôtel La Fabrique offers accommodation in La R...,46.156155,-1.148583,46.159113,-1.152043,19.45650,4.64


In [26]:
# Identification of the information contained in the weather column for the 1st line 
print(df_s3['Weather'].iloc[0])

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1721001600, 'main': {'temp': 289.08, 'feels_like': 288.79, 'temp_min': 287.26, 'temp_max': 289.08, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 1006, 'humidity': 79, 'temp_kf': 1.82}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 52}, 'wind': {'speed': 2.63, 'deg': 159, 'gust': 2.79}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2024-07-15 00:00:00'}, {'dt': 1721012400, 'main': {'temp': 287.72, 'feels_like': 287.3, 'temp_min': 286.59, 'temp_max': 287.72, 'pressure': 1011, 'sea_level': 1011, 'grnd_level': 1005, 'humidity': 79, 'temp_kf': 1.13}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 82}, 'wind': {'speed': 2.93, 'deg': 137, 'gust': 3.81}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2024-07-15 03:00:00'}, {'dt': 1721023200, 'main': {'temp': 289.04, 'feels_like'

In [27]:
# Function to extract data from weather column
def extract_combined_weather_features(row):
    try:
        # Attempts to convert weather string to JSON, otherwise converts to dictionary
        if isinstance(row['Weather'], str) and row['Weather'].strip().startswith('{'):
            try:
                weather_data = json.loads(row['Weather'])
            except json.JSONDecodeError:
                weather_data = ast.literal_eval(row['Weather'].replace("'", '"'))
        else:
            weather_data = ast.literal_eval(row['Weather'])
        
        if 'daily' in weather_data:
            daily_data = weather_data['daily'] # Access daily data
        
            # Calculations for weather measurements to extract
            max_temp = max(day['temp']['max'] for day in daily_data)
            min_temp = min(day['temp']['min'] for day in daily_data)
            avg_temp = sum(day['temp']['day'] for day in daily_data) / len(daily_data)
            avg_humidity = sum(day['humidity'] for day in daily_data) / len(daily_data)
            weather_descriptions = ', '.join(set(day['weather'][0]['description'] for day in daily_data))
            avg_wind_speed = sum(day['wind_speed'] for day in daily_data) / len(daily_data)
            max_uv = max(day['uvi'] for day in daily_data)
            total_precip = sum(day.get('rain', 0) for day in daily_data)
            avg_pressure = sum(day['pressure'] for day in daily_data) / len(daily_data)
            
            # Returns all calculated measurements as a series
            return pd.Series([max_temp, min_temp, avg_temp, avg_humidity, weather_descriptions, avg_wind_speed, max_uv, total_precip, avg_pressure])
        else:
            # If 'daily' key doesn't exist, return None for all attributes
            return pd.Series([None] * 9)
    except Exception as e:
        print(f"Error processing row: {e}")
        return pd.Series([None] * 9) # Return None for each attribute on error

# Assuming df_s3 is your DataFrame
# Applying the function on df_s3 to enrich the data
df_s3[['MaxTemp', 'MinTemp', 'AvgTemp', 'AvgHumidity', 'WeatherDesc', 'AvgWindSpeed', 'MaxUV', 'TotalPrecip', 'AvgPressure']] = df_s3.apply(extract_combined_weather_features, axis=1)

In [28]:
# Show the first rows to check for new data
df_s3.head()

,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,Weather,...,TotalPrecipitation,MaxTemp,MinTemp,AvgTemp,AvgHumidity,WeatherDesc,AvgWindSpeed,MaxUV,TotalPrecip,AvgPressure
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",...,8.34,None,None,None,None,None,None,None,None,None
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",...,8.34,None,None,None,None,None,None,None,None,None
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",...,8.34,None,None,None,None,None,None,None,None,None
3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",...,8.34,None,None,None,None,None,None,None,None,None
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.51146,"{'cod': '200', 'message': 0, 'cnt': 40, 'list'...",...,8.34,None,None,None,None,None,None,None,None,None


In [29]:
# creation of the final dataframe df_final_s3
df_final_s3= df_s3

In [30]:
# saving final s3 data to a csv file
df_full.to_csv('df_final_s3.csv', index=False)

In [33]:
# crée un moteur de base de données SQLite en mémoire
engine = create_engine('sqlite:///:memory:', echo=True)

In [34]:
df_final_s3.to_sql("hotels", engine)

2024-07-15 00:14:03,286 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 00:14:03,289 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotels")
2024-07-15 00:14:03,289 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 00:14:03,290 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotels")
2024-07-15 00:14:03,290 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 00:14:03,292 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotels (
	"index" BIGINT, 
	"City" TEXT, 
	"Hotel Name" TEXT, 
	"Hotel URL" TEXT, 
	"Score" FLOAT, 
	"Description" TEXT, 
	lat_hotels FLOAT, 
	lon_hotels FLOAT, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT, 
	"Weather" TEXT, 
	"AverageTemp" FLOAT, 
	"TotalPrecipitation" FLOAT, 
	"MaxTemp" TEXT, 
	"MinTemp" TEXT, 
	"AvgTemp" TEXT, 
	"AvgHumidity" TEXT, 
	"WeatherDesc" TEXT, 
	"AvgWindSpeed" TEXT, 
	"MaxUV" TEXT, 
	"TotalPrecip" TEXT, 
	"AvgPressure" TEXT
)


2024-07-15 00:14:03,292 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2024-07-15 00:14:03,2

876

In [35]:
# Connexion à la base
conn = engine.connect()

In [36]:
data = text("SELECT * FROM hotels")

In [37]:
# récupérer les résultats sous forme de DataFrame.
pd.read_sql_query(data, conn).head()

2024-07-15 00:14:46,946 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 00:14:46,947 INFO sqlalchemy.engine.Engine SELECT * FROM hotels
2024-07-15 00:14:46,947 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()


,index,City,Hotel Name,Hotel URL,Score,Description,lat_hotels,lon_hotels,Latitude,Longitude,...,TotalPrecipitation,MaxTemp,MinTemp,AvgTemp,AvgHumidity,WeatherDesc,AvgWindSpeed,MaxUV,TotalPrecip,AvgPressure
0,0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617,48.635954,-1.51146,...,8.34,None,None,None,None,None,None,None,None,None
1,1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,"Composed of 2 different buildings, Les Terrass...",48.635349,-1.510379,48.635954,-1.51146,...,8.34,None,None,None,None,None,None,None,None,None
2,2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,This Mercure is situated in parkland just 2 km...,48.614247,-1.510545,48.635954,-1.51146,...,8.34,None,None,None,None,None,None,None,None,None
3,3,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.1,Le Relais Du Roy is a 3-star hotel situated ne...,48.616263,-1.510906,48.635954,-1.51146,...,8.34,None,None,None,None,None,None,None,None,None
4,4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396,48.635954,-1.51146,...,8.34,None,None,None,None,None,None,None,None,None
